In [1]:
from core.llm.llm import LLM
from core.message.message import UserMessage , SystemMessage,LLMMessage,ToolMessage
from core.memory.simple_memory import SimpleMemory
from core.tools.tool_manager import ToolManager
from rich import print as rprint



In [2]:


    
#初始化对话历史
memory = SimpleMemory(system_message=SystemMessage(content = "你是一个暴躁的编程助手，喜欢用反问句回答问题"),
                history=[])



In [3]:

#初始化LLM
llm = LLM(model="deepseek-chat")


In [4]:


#初始化工具服务

toolmanager = ToolManager("tools/mcp/config.json")


In [5]:


#user第一次的prompt输入
user_input =  input()
memory.add(UserMessage(content=user_input))


In [6]:


# 3. Agent 决策循环 (简易版)
while True:
    #LLM 生成回复 
    native_response = llm.chat(memory.to_messages())
    
    print(native_response)
    response = native_response.choices[0].message
    memory.add(LLMMessage(content=response.content,tool_calls=response.tool_calls))
    
    # 如果 LLM 不需要调用工具，直接输出结果并停止
    if response.tool_calls == None:
        print("\nAgent 回复:", response.content)
        #下一次输入
        user_input =  input()
        memory.add(UserMessage(content=user_input))
        continue

    # 4. 如果 LLM 需要调用工具
    
    for content_block in response.content:
        if content_block.type == "tool_use":
            tool_name = content_block.name
            tool_args = content_block.input
            tool_use_id = content_block.id

            # 执行 MCP 工具
            result = ToolManager.handle_tool_call(tool_name, tool_args)
            
            # 将结果反馈给 LLM (处理 result 可能包含 text 或 image 的情况)
            tool_result_content = []
            for item in result:
                if hasattr(item, 'text'):
                    tool_result_content.append({"type": "text", "text": item.text})

            memory.Add(
                ToolMessage(content=tool_result_content,
                                tool_use_id=tool_use_id))
            print(f"工具执行完毕，正在发回 LLM...")


[
    SystemMessage(role='system', content='你是一个暴躁的编程助手，喜欢用反问句回答问题'),
    UserMessage(role='user', content='你好，介绍一下你自己')
]

{
    'model': 'deepseek-chat',
    'messages': [
        {'role': 'system', 'content': '你是一个暴躁的编程助手，喜欢用反问句回答问题'},
        {'role': 'user', 'content': '你好，介绍一下你自己'}
    ],
    'stream': True
}

ChatCompletionChunk(id='b5f2a79c-ddba-49f7-838f-02fa14c6dbc3', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1766659567, model='deepseek-chat', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_eaab8d114b_prod0820_fp8_kvcache', usage=None)ChatCompletionChunk(id='b5f2a79c-ddba-49f7-838f-02fa14c6dbc3', choices=[Choice(delta=ChoiceDelta(content='你', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1766659567, model='deepseek-chat', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_eaab8d114b_prod0820_fp8_kvcache', usage=None)ChatCompletionChunk(id='b5f2a79c-ddba-49f7-838f-02fa14c6dbc3', choices=[Choice(delta=ChoiceDelta(content='眼睛', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1766

Last Msg: [SystemMessage(role='system', content='你是一个暴躁的编程助手，喜欢用反问句回答问题'), 
UserMessage(role='user', content='你好，介绍一下你自己')]

role='assistant' content='对不起，我遇到了一点错误。' tool_calls=None


AttributeError: 'LLMMessage' object has no attribute 'choices'